<a href="https://colab.research.google.com/github/CodeHunterOfficial/CCPiOIIC/blob/main/ML/%D0%90%D1%80%D0%B0%D0%B1%D0%BE%D0%B2%D0%9C%D0%9A_%D0%A0%D0%B5%D0%B0%D0%BB%D0%B8%D0%B7%D0%B0%D1%86%D0%B8%D1%8F_%D1%8F%D0%B7%D1%8B%D0%BA%D0%BE%D0%B2%D0%BE%D0%B9_%D0%BC%D0%BE%D0%B4%D0%B5%D0%BB%D0%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Генерация текста

В этом уроке мы познакомимся с генерацией текста на практике, а именно -- реализуем языковую модель (то есть обучим модель генерировать текст заданном стиле). В качестве модели возьмём Char-RNN, то есть работать будем на уровне отдельных символов (букв).

В качестве обучающей выборки (корпуса) возьмём текст произведения Шекспира.
В результате наша обученная языковая модель должна генерировать текст в таком же стиле.

### Используем TensorFlow 2.0

На момент подготовки этих материалов в Google Colab по умолчанию используется версия TensorFlow 1.X

Переключаемся на версию 2.0 (работает только в Colab)

In [1]:
%tensorflow_version 2.x

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


### Загрузка библиотек
TensorFlow должен иметь как минимум версию 2.0

In [2]:
import codecs
import numpy as np
import tensorflow as tf
print(tf.__version__)

2.17.1


### Загрузка датасета

Скачиваем файл с текстом (`shakespeare.txt`) и загружаем его содержимое в переменную `text`.

Посмотрим, как выглядит текст, распечатав его фрагмент. Видно, что тексту свойственна некоторая стилистика пьесы.

In [3]:
data_fpath = tf.keras.utils.get_file(
    'shakespeare.txt',
    'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

text = codecs.open(data_fpath, 'r', encoding='utf8').read()
print(text[:250])

1115394/1115394 ━━━━━━━━━━━━━━━━━━━━ 1s 1us/step
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



### Конвертация символов в индексы

Как и раньше (как мы делали в случае классификации текстов), нам будет удобно работать с некоторым словарём и индексами слов в данном словаре. Только сейчас вместо слов мы будем испоьзовать символы (буквы итд).

Чтобы получить словарь `vocab` достаточно применить оператор `set` к нашему тексту (то есть конвертировать последовательность всех символов в множество = удалить все повторы). `VOCAB_SIZE` -- количество элементов в словаре.

Затем, как и раньше, создаём отображения символа в индекс и наоборот (`char2idx`, `idx2char`)

Теперь конвертируем наш текст в последовательность индексов с помощью `char2idx`

In [4]:
vocab = sorted(set(text))
VOCAB_SIZE = len(vocab)

print('Vocab: {}'.format(vocab))
print('{} unique characters'.format(VOCAB_SIZE))

char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

print('Example of the encoded text: {}'.format(text_as_int[:13]))

Vocab: ['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
65 unique characters
Example of the encoded text: [18 47 56 57 58  1 15 47 58 47 64 43 52]


### Подготовка датасета

Во время инференса наш генератор  будет работать следующим образом: сначала мы подаём входной символ или входную последовательность (зерно), получаем первый выходной символ, а затем подаём его как входной символ и так далее (генерируем по одному символу за раз).

А во время обучения будем обучать модель работать сразу с целой последовательностью. Например, обученная модель должна по входу `[F, i, r, s, ...]` выдавать `[i, r, s, t, ...]`. То есть ту же последовательность, но сдвинутую на 1 элемент. В данном случае это будет эквивалентно Many-to-Many Sync (только во время обучения).

Таким образом, зафиксируем длину рабочей цепочки `SEQ_LEN` (на которой будем обучаться), разрежем весь текст на цепочки длины `SEQ_LEN+1` (остаток отбросим), и из каждой такой цепочки длины `SEQ_LEN+1` получим пару цепочек длины `SEQ_LEN`, сдвинутых на 1 элемент: входная цепочка (без последнего элемента) и целевая (истинная) цепочка (начиная со второго элемента).

Кроме того, зафиксируем размер батча для обучения и отбросим цепочки в конце, которые не могут наполнить полный батч (чтоб кол-во обучающих цепочек было кратно размеру батча).


In [5]:
SEQ_LEN = 1000
BATCH_SIZE = 64

input_seqs = []
target_seqs = []

num_seqs = len(text_as_int) // (SEQ_LEN+1)
for i in range(num_seqs):
    seq = text_as_int[i:i+SEQ_LEN+1]
    input_seqs.append(np.array(seq[:-1]))
    target_seqs.append(np.array(seq[1:]))

input_seqs = np.array(input_seqs)
target_seqs = np.array(target_seqs)

input_seqs = input_seqs[:(len(input_seqs)//BATCH_SIZE)*BATCH_SIZE]
target_seqs = target_seqs[:(len(input_seqs)//BATCH_SIZE)*BATCH_SIZE]

print('Input: {} ...'.format([idx2char[i] for i in input_seqs[0][:15]]))
print('Target: {} ...'.format([idx2char[i] for i in target_seqs[0][:15]]))

Input: ['F', 'i', 'r', 's', 't', ' ', 'C', 'i', 't', 'i', 'z', 'e', 'n', ':', '\n'] ...
Target: ['i', 'r', 's', 't', ' ', 'C', 'i', 't', 'i', 'z', 'e', 'n', ':', '\n', 'B'] ...


### Функция построения модели

Здесь мы создадим нашу модель с помощью `tf.keras.Sequential`.

Модель будет состоять из трёх слоёв:
* Embedding слой для отображения индексов букв в их вектрное представление
* Рекуррентный слой GRU
* Полносвязный слой, предсказывающий распределение по различным буквам (например, можно потом навесить Argmax, чтобы понять, какую именно букву предсказывает слой)

На выходе нам нужно получить последовательность такой же длины, что и входная,то есть чтоб GRU слой возвращал всю цепочку скрытых векторов, а не только последний. Для этого в параметрах GRU слоя зададим `return_sequences=True`.

Во время обучения будем просить модель предсказать входную цепочку, сдвинутую на 1 элемент. A во время инференса -- будем постепенно подавать символ за символом (то есть при каждом инференсе модель будет делать отображение "один в один"). Теоретически можно было бы и во время обучения делать так же -- постепенно генерировать выход (символ за символом), подавая результат предыдущей итерации (предыдущий символ) на вход в текущей итерации. Но сеть будет обучаться лучше, если мы будем "заставлять" её использовать "правильные" входы, а не то, что она сама нагенерировала.

Чтобы во время инференса при различных запусках модели она помнила своё предыдущее состояние (иначе не получится использовать рекуррентность, ведь мы будем подавать последовательно последовательности из одного элемента), мы укажем флаг `stateful=True`

Кроме того, если модель имеет флаг `stateful=True`, ей нужно заранее знать размер батча (зададим через `batch_input_shape`). А так как у нас размер батча будет разный (для обучения >1, для инференса =1), нам понадобится создать модель два раза. Чтобы не дублировать код создания модели, обернём её создание в функцию `build_model`, которая принимает размер батча в качестве аргумента.

In [7]:
def build_model(batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(VOCAB_SIZE, 256, input_length=None),  # Используем input_length вместо batch_input_shape
        tf.keras.layers.GRU(256, return_sequences=True, stateful=True),
        tf.keras.layers.Dense(VOCAB_SIZE),
    ])
    model.build(input_shape=(batch_size, None))  # Указываем input_shape при вызове build
    return model

model = build_model(BATCH_SIZE)

### Обучение модели

На выходе модели у нас полносвязный слой без функции активации, то есть это просто логиты. По ним нам надо по сути сделать классификацию (номер класса = номер предсказанного символа). Для логитов и целевых (истинных) значений, представленных индексами, надо использовать лосс `SparseCategoricalCrossentropy(from_logits=True)`

Далее обучаем модель стандартным способом на наборах `(input_seqs, target_seqs)`

In [8]:
EPOCHS = 100

loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer='adam', loss=loss)

history = model.fit(input_seqs,
    target_seqs,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE)

Epoch 1/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 5s 87ms/step - loss: 3.6071
Epoch 2/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 81ms/step - loss: 2.3486
Epoch 3/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 82ms/step - loss: 1.9888
Epoch 4/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 81ms/step - loss: 1.6831
Epoch 5/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 81ms/step - loss: 1.3645
Epoch 6/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 82ms/step - loss: 1.0243
Epoch 7/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 81ms/step - loss: 0.7055
Epoch 8/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 82ms/step - loss: 0.4543
Epoch 9/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 81ms/step - loss: 0.2836
Epoch 10/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 82ms/step - loss: 0.1859
Epoch 11/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 81ms/step - loss: 0.1248
Epoch 12/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 82ms/step - loss: 0.0909
Epoch 13/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 82ms/step - loss: 0.0740
Epoch 14/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 81ms/step - loss: 0.0608
Epoch 15/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 82ms/step - lo

### Создание модели для инференса

После обучения модели нам надо создать такую же модель, но с размером батча =1, которую позже будем использовать для инференса (`model_inf`). Воспользуемся для этого функцией `build_model`, а затем скопируем обученные веса из `model` в новую модель `model_inf`.

In [9]:
model_inf = build_model(1)

for i in range(len(model_inf.layers)):
    for j in range(len(model_inf.layers[i].weights)):
        model_inf.layers[i].weights[j].assign(model.layers[i].weights[j])

### Функция генерации текста

Создадим функцию, генерирующую текст по данной модели (`model`), входной цепочке (зерну `seed`) и желаемому количеству сгенерированных символов (`out_len`).

Внутри функции сначчала вызовем `model.reset_states()` для обнуления предыдущей истории состояния.

Затем конвертируем входной текст `seed` (символы в индексы) и прогоним его через нашу модель. Нас интересует результат, соответствующих только последнему выходу (символу).

Как получить сам символ по предсказанию модели? Один из простейших способов -- просто взять argmax от предсказанных логитов (взять символ с наибольшей вероятностью). Однако, в таком случае выход часто будет слишком предсказуемым и иногда цепочка будет застрявать в цикличности (повторяться).

Для того, чтобы сделать предсказание менее предсказуемым, воспользуемся реальным распределенем и будем "сэмплировать" из него (то есть выбирать символ случайным образом в соответствии с его вероятностью). Сделать это можно с помощью функции `tf.random.categorical`. На вход в `categorical` мы подаём всю матрицу `pred` (у которой первое измерение это длина цепочки, а второе - распределение по классам), а на выходе получаем список сэмплов, по одному для каждого элемента последовательности). А так как нас интересует лишь последний символ, необходимо выбрать только его с помощью индекса `[-1]`.

Кроме того, можно ввести параметр (так называемая `температура`), с помощью которого можно управлять выходным распределением и таким образом влиять на непредсказуемость результата. Чем выше температура, тем более случайным будет предсказанный символ. Например, если мы разделим логиты на большое число (температуру), то если бы мы применили софтмакс (для получения вероятностей), то распределение стремилось бы к равномерному (у разных классов уравниваются шансы).

Всё, что мы рассмотрели выше, соответствует первой итерации цикла `for`. Далее процесс повторяется, но теперь на входе каждый раз цепочка из одного символа (сгенерированного на предыдущей итерации). И делаем так, пока не соберем выходную цепочку длины `out_len`.

In [15]:
def generate_text(model, seed, out_len, temperature=1.0):
    text_generated = []

    # Обнуляем состояние GRU слоя
    for layer in model.layers:
        if hasattr(layer, 'reset_states'):
            layer.reset_states()

    # Конвертируем входную цепочку в индексы
    inp = np.array([char2idx[s] for s in seed])

    for i in range(out_len):
        # Получаем предсказания для входной цепочки inp
        # pred - матрица размерности (длина цепочки, распределение по классам)
        # На первой итерации цикла длина цепочки равна длине seed, а затем длина равна 1
        pred = model(inp[None, ...])[0]

        # Применяем температуру к предсказаниям
        pred = pred / temperature

        # Сэмплируем символ из распределения
        pred_c = tf.random.categorical(pred, num_samples=1)[-1][0].numpy()

        text_generated.append(idx2char[pred_c])

        # Новый вход -- только что сгенерированный символ
        inp = np.array([pred_c])

    return (seed + ''.join(text_generated))

### Запуск генератора текста

Запускаем генерацию текста, передавая на вход желаемое начало цепочки `seed`.

In [16]:
print(generate_text(model_inf, seed=u"MONTAGUE:", out_len=500))

MONTAGUE:
Against him first: he's a very dog to the commonalty.

Second Citizen:
Consider you what services he has done for his country?

First Citizen:
Very well; anaccounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance;


**[Задание 1]** Поэксперементируйте с параметром `температура`. Найдите примеры значения параметра, при которых модель ведёт себя предсказуемо (детерминировано) и не предсказуемо (случайно).




Параметр **температура** в генерации текста управляет степенью случайности предсказаний модели. Чем выше температура, тем более случайным будет выход модели, а чем ниже температура, тем более детерминированным и предсказуемым будет результат. Давайте поэкспериментируем с разными значениями температуры и посмотрим, как это влияет на генерацию текста.



### **1. Низкая температура (например, 0.1)**
При низкой температуре модель становится более уверенной в своих предсказаниях и выбирает наиболее вероятные символы. Это делает текст более предсказуемым и связным, но иногда может приводить к повторениям или зацикливанию.



In [18]:
print(generate_text(model_inf, seed=u"MONTAGUE:", out_len=100, temperature=0.1))

MONTAGUE:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First,



### **2. Средняя температура (например, 1.0)**
При температуре 1.0 модель использует исходное распределение вероятностей, что приводит к балансу между случайностью и предсказуемостью.


In [19]:
print(generate_text(model_inf, seed=u"MONTAGUE:", out_len=100, temperature=1.0))

MONTAGUE:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First,



### **3. Высокая температура (например, 2.0)**
При высокой температуре модель становится более случайной, выбирая даже маловероятные символы. Это может привести к бессвязному или абсурдному тексту.



In [20]:
print(generate_text(model_inf, seed=u"MONTAGUE:", out_len=100, temperature=2.0))

MONTAGUE:
Resolved. RHSe; for the alved ratinok; $fwe know't.

FirstcMluplKTPiut, we know Caius Marciusl?

Al



### **4. Очень высокая температура (например, 5.0)**
При очень высокой температуре модель становится полностью случайной, и текст теряет всякую связность.



In [21]:
print(generate_text(model_inf, seed=u"MONTAGUE:", out_len=100, temperature=5.0))

MONTAGUE:
sw?RRkLe,pe'Bevee:X VDrt'bfyuE?

iruF!G
bYqH.X!buIXWelFr,w-KtTsy
sVJKqSDqTEVerzj
hu'Yl!Qwtoqy&
OpE!


**[Задание 2]** Попробуйте улучшить нейросеть за счёт увеличения её глубины и добавления регуляризации (dropout).



In [22]:
# Переключаемся на TensorFlow 2.x (если используем Google Colab)
%tensorflow_version 2.x

# Импорт библиотек
import codecs
import numpy as np
import tensorflow as tf
print(tf.__version__)

# Загрузка датасета
data_fpath = tf.keras.utils.get_file(
    'shakespeare.txt',
    'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

text = codecs.open(data_fpath, 'r', encoding='utf8').read()
print(text[:250])

# Создание словаря символов
vocab = sorted(set(text))
VOCAB_SIZE = len(vocab)

print('Vocab: {}'.format(vocab))
print('{} unique characters'.format(VOCAB_SIZE))

# Создание отображений символов в индексы и обратно
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

# Конвертация текста в последовательность индексов
text_as_int = np.array([char2idx[c] for c in text])

print('Example of the encoded text: {}'.format(text_as_int[:13]))

# Подготовка датасета
SEQ_LEN = 1000
BATCH_SIZE = 64

input_seqs = []
target_seqs = []

num_seqs = len(text_as_int) // (SEQ_LEN+1)
for i in range(num_seqs):
    seq = text_as_int[i:i+SEQ_LEN+1]
    input_seqs.append(np.array(seq[:-1]))
    target_seqs.append(np.array(seq[1:]))

input_seqs = np.array(input_seqs)
target_seqs = np.array(target_seqs)

input_seqs = input_seqs[:(len(input_seqs)//BATCH_SIZE)*BATCH_SIZE]
target_seqs = target_seqs[:(len(input_seqs)//BATCH_SIZE)*BATCH_SIZE]

print('Input: {} ...'.format([idx2char[i] for i in input_seqs[0][:15]]))
print('Target: {} ...'.format([idx2char[i] for i in target_seqs[0][:15]]))

# Функция построения модели
def build_model(batch_size):
    model = tf.keras.Sequential([
        # Embedding слой для отображения индексов символов в векторное представление
        tf.keras.layers.Embedding(VOCAB_SIZE, 256, input_length=None),

        # Первый GRU слой с Dropout
        tf.keras.layers.GRU(512, return_sequences=True, stateful=True),
        tf.keras.layers.Dropout(0.2),  # Добавляем Dropout для регуляризации

        # Второй GRU слой с Dropout
        tf.keras.layers.GRU(512, return_sequences=True, stateful=True),
        tf.keras.layers.Dropout(0.2),  # Добавляем Dropout для регуляризации

        # Полносвязный слой для предсказания следующего символа
        tf.keras.layers.Dense(VOCAB_SIZE),
    ])
    model.build(input_shape=(batch_size, None))  # Указываем input_shape при вызове build
    return model

# Создаём модель для обучения
model = build_model(BATCH_SIZE)

# Компилируем модель
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer='adam', loss=loss)

# Обучаем модель
EPOCHS = 20  # Увеличиваем количество эпох для более глубокой модели
history = model.fit(input_seqs,
                    target_seqs,
                    epochs=EPOCHS,
                    batch_size=BATCH_SIZE)

# Создаём модель для инференса
model_inf = build_model(1)

# Копируем веса из обученной модели
for i in range(len(model_inf.layers)):
    for j in range(len(model_inf.layers[i].weights)):
        model_inf.layers[i].weights[j].assign(model.layers[i].weights[j])

# Функция генерации текста
def generate_text(model, seed, out_len, temperature=1.0):
    text_generated = []

    # Обнуляем состояние GRU слоя
    for layer in model.layers:
        if hasattr(layer, 'reset_states'):
            layer.reset_states()

    # Конвертируем входную цепочку в индексы
    inp = np.array([char2idx[s] for s in seed])

    for i in range(out_len):
        # Получаем предсказания для входной цепочки inp
        pred = model(inp[None, ...])[0]

        # Применяем температуру к предсказаниям
        pred = pred / temperature

        # Сэмплируем символ из распределения
        pred_c = tf.random.categorical(pred, num_samples=1)[-1][0].numpy()

        text_generated.append(idx2char[pred_c])

        # Новый вход -- только что сгенерированный символ
        inp = np.array([pred_c])

    return (seed + ''.join(text_generated))

# Генерация текста с использованием модифицированной модели
print(generate_text(model_inf, seed=u"MONTAGUE:", out_len=500, temperature=0.8))

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
2.17.1
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

Vocab: ['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
65 unique characters
Example of the encoded text: [18 47 56 57 58  1 15 47 58 47 64 43 52]
Input: ['F', 'i', 'r', 's', 't', ' ', 'C', 'i', 't', 'i', 'z', 'e', 'n', ':', '\n'] ...
Target: ['i', 'r', 's', 't', ' ', 'C', 'i', 't', 'i', 'z', 'e', 'n', ':', '\n', 'B'] ...
Epoch 1/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 10s 425ms/step - loss: 3.5821
E


**[Задание 3]** Обучите модель Char-RNN на другом корпусе: возьмите датасет IMDB (из предыдущего модуля), объедените все отзывы в один текст и обучитесь на нём.



In [ ]:
# Импорт библиотек
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Загрузка датасета IMDB
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=10000)

# Функция для преобразования последовательностей обратно в текст
def decode_review(sequence, word_index):
    reverse_word_index = {v: k for k, v in word_index.items()}
    return ' '.join([reverse_word_index.get(i, '?') for i in sequence])

# Загрузка словаря слов IMDB
word_index = imdb.get_word_index()

# Объединение всех отзывов в один текст
text = ' '.join([decode_review(review, word_index) for review in x_train + x_test])

# Вывод фрагмента текста
print(text[:1000])

# Создание словаря символов
vocab = sorted(set(text))
VOCAB_SIZE = len(vocab)

print('Vocab: {}'.format(vocab))
print('{} unique characters'.format(VOCAB_SIZE))

# Создание отображений символов в индексы и обратно
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

# Конвертация текста в последовательность индексов
text_as_int = np.array([char2idx[c] for c in text])

print('Example of the encoded text: {}'.format(text_as_int[:13]))

# Подготовка датасета
SEQ_LEN = 1000
BATCH_SIZE = 64

input_seqs = []
target_seqs = []

num_seqs = len(text_as_int) // (SEQ_LEN+1)
for i in range(num_seqs):
    seq = text_as_int[i:i+SEQ_LEN+1]
    input_seqs.append(np.array(seq[:-1]))
    target_seqs.append(np.array(seq[1:]))

input_seqs = np.array(input_seqs)
target_seqs = np.array(target_seqs)

input_seqs = input_seqs[:(len(input_seqs)//BATCH_SIZE)*BATCH_SIZE]
target_seqs = target_seqs[:(len(input_seqs)//BATCH_SIZE)*BATCH_SIZE]

print('Input: {} ...'.format([idx2char[i] for i in input_seqs[0][:15]]))
print('Target: {} ...'.format([idx2char[i] for i in target_seqs[0][:15]]))

# Функция построения модели
def build_model(batch_size):
    model = tf.keras.Sequential([
        # Embedding слой для отображения индексов символов в векторное представление
        tf.keras.layers.Embedding(VOCAB_SIZE, 256, input_length=None),

        # Первый GRU слой с Dropout
        tf.keras.layers.GRU(512, return_sequences=True, stateful=True),
        tf.keras.layers.Dropout(0.2),  # Добавляем Dropout для регуляризации

        # Второй GRU слой с Dropout
        tf.keras.layers.GRU(512, return_sequences=True, stateful=True),
        tf.keras.layers.Dropout(0.2),  # Добавляем Dropout для регуляризации

        # Полносвязный слой для предсказания следующего символа
        tf.keras.layers.Dense(VOCAB_SIZE),
    ])
    model.build(input_shape=(batch_size, None))  # Указываем input_shape при вызове build
    return model

# Создаём модель для обучения
model = build_model(BATCH_SIZE)

# Компилируем модель
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer='adam', loss=loss)

# Обучаем модель
EPOCHS = 20  # Увеличиваем количество эпох для более глубокой модели
history = model.fit(input_seqs,
                    target_seqs,
                    epochs=EPOCHS,
                    batch_size=BATCH_SIZE)

# Создаём модель для инференса
model_inf = build_model(1)

# Копируем веса из обученной модели
for i in range(len(model_inf.layers)):
    for j in range(len(model_inf.layers[i].weights)):
        model_inf.layers[i].weights[j].assign(model.layers[i].weights[j])

# Функция генерации текста
def generate_text(model, seed, out_len, temperature=1.0):
    text_generated = []

    # Обнуляем состояние GRU слоя
    for layer in model.layers:
        if hasattr(layer, 'reset_states'):
            layer.reset_states()

    # Конвертируем входную цепочку в индексы
    inp = np.array([char2idx[s] for s in seed])

    for i in range(out_len):
        # Получаем предсказания для входной цепочки inp
        pred = model(inp[None, ...])[0]

        # Применяем температуру к предсказаниям
        pred = pred / temperature

        # Сэмплируем символ из распределения
        pred_c = tf.random.categorical(pred, num_samples=1)[-1][0].numpy()

        text_generated.append(idx2char[pred_c])

        # Новый вход -- только что сгенерированный символ
        inp = np.array([pred_c])

    return (seed + ''.join(text_generated))

# Генерация текста с использованием модифицированной модели
print(generate_text(model_inf, seed=u"I think this movie is", out_len=500, temperature=0.8))

17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
1641221/1641221 ━━━━━━━━━━━━━━━━━━━━ 1s 1us/step
the as you with out themselves powerful lets loves their becomes reaching had journalist of lot from anyone to have after out atmosphere never more room and it so heart shows to years of every never going and help moments or of every chest visual movie except her was several of enough more with is now current film as you of mine potentially unfortunately of you than him that with out themselves her get for was camp of you movie sometimes movie that with scary but and to story wonderful that in seeing in character to of 70s musicians with heart had shadows they of here that with her serious to have does when from why what have critics they is you that isn't one will very to as itself with other and in of seen over landed for anyone of and br show's to whether from than out themselves history he name half some br of and odd was two most of mean for 1 any an boat she he should is thought fr

**[Задание 4]** Обучите модель синтеза текста на отдельных словах а не на символах. Используйте для этого датасет IMDB с ограниченным словарём `(num_words=...)`.


In [ ]:
# Импорт библиотек
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Загрузка датасета IMDB с ограниченным словарём
NUM_WORDS = 1000  # Ограничиваем словарь
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=NUM_WORDS)

# Загрузка словаря слов IMDB
word_index = imdb.get_word_index()

# Создание обратного словаря (индекс -> слово)
reverse_word_index = {v: k for k, v in word_index.items()}

# Функция для преобразования последовательности индексов в текст
def decode_review(sequence):
    return ' '.join([reverse_word_index.get(i, '?') for i in sequence])

# Объединение всех отзывов в один список слов
words = []
for review in x_train + x_test:
    words.extend(review)

# Вывод фрагмента текста
print(decode_review(words[:50]))

# Создание словаря слов
vocab = sorted(set(words))
VOCAB_SIZE = len(vocab)

print('Vocab size: {}'.format(VOCAB_SIZE))

# Создание отображений слов в индексы и обратно
word2idx = {u:i for i, u in enumerate(vocab)}
idx2word = np.array(vocab)

# Конвертация текста в последовательность индексов
words_as_int = np.array([word2idx[word] for word in words])

print('Example of the encoded text: {}'.format(words_as_int[:10]))

# Подготовка датасета
SEQ_LEN = 100  # Длина последовательности слов
BATCH_SIZE = 64

input_seqs = []
target_seqs = []

num_seqs = len(words_as_int) // (SEQ_LEN+1)
for i in range(num_seqs):
    seq = words_as_int[i:i+SEQ_LEN+1]
    input_seqs.append(np.array(seq[:-1]))
    target_seqs.append(np.array(seq[1:]))

input_seqs = np.array(input_seqs)
target_seqs = np.array(target_seqs)

input_seqs = input_seqs[:(len(input_seqs)//BATCH_SIZE)*BATCH_SIZE]
target_seqs = target_seqs[:(len(input_seqs)//BATCH_SIZE)*BATCH_SIZE]

print('Input: {} ...'.format([idx2word[i] for i in input_seqs[0][:10]]))
print('Target: {} ...'.format([idx2word[i] for i in target_seqs[0][:10]]))

# Функция построения модели
def build_model(batch_size):
    model = tf.keras.Sequential([
        # Embedding слой для отображения индексов слов в векторное представление
        tf.keras.layers.Embedding(VOCAB_SIZE, 256, input_length=None),

        # Первый GRU слой с Dropout
        tf.keras.layers.GRU(512, return_sequences=True, stateful=True),
        tf.keras.layers.Dropout(0.2),  # Добавляем Dropout для регуляризации

        # Второй GRU слой с Dropout
        tf.keras.layers.GRU(512, return_sequences=True, stateful=True),
        tf.keras.layers.Dropout(0.2),  # Добавляем Dropout для регуляризации

        # Полносвязный слой для предсказания следующего слова
        tf.keras.layers.Dense(VOCAB_SIZE),
    ])
    model.build(input_shape=(batch_size, None))  # Указываем input_shape при вызове build
    return model

# Создаём модель для обучения
model = build_model(BATCH_SIZE)

# Компилируем модель
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer='adam', loss=loss)

# Обучаем модель
EPOCHS = 10  # Увеличиваем количество эпох для более глубокой модели
history = model.fit(input_seqs,
                    target_seqs,
                    epochs=EPOCHS,
                    batch_size=BATCH_SIZE)

# Создаём модель для инференса
model_inf = build_model(1)

# Копируем веса из обученной модели
for i in range(len(model_inf.layers)):
    for j in range(len(model_inf.layers[i].weights)):
        model_inf.layers[i].weights[j].assign(model.layers[i].weights[j])

# Функция генерации текста
def generate_text(model, seed, out_len, temperature=1.0):
    text_generated = []

    # Обнуляем состояние GRU слоя
    for layer in model.layers:
        if hasattr(layer, 'reset_states'):
            layer.reset_states()

    # Конвертируем входную цепочку в индексы
    inp = np.array([word2idx[word] for word in seed.split()])

    for i in range(out_len):
        # Получаем предсказания для входной цепочки inp
        pred = model(inp[None, ...])[0]

        # Применяем температуру к предсказаниям
        pred = pred / temperature

        # Сэмплируем слово из распределения
        pred_c = tf.random.categorical(pred, num_samples=1)[-1][0].numpy()

        text_generated.append(idx2word[pred_c])

        # Новый вход -- только что сгенерированное слово
        inp = np.append(inp, pred_c)[-SEQ_LEN:]  # Сохраняем только последние SEQ_LEN слов

    return (seed + ' ' + ' '.join(text_generated))

# Генерация текста с использованием модифицированной модели
print(generate_text(model_inf, seed="I think this movie is", out_len=50, temperature=0.8))